# 추론 및 테스트 기반 pseudo data 생성

In [ ]:
from ultralytics import YOLO
import pandas as pd
import os
from datetime import datetime

In [ ]:
model_path = '/your model path'

# 모델 로드
model = YOLO(model_path)

In [ ]:
# 테스트 세트 경로
test_images_path = '/your_test_img_path'  # 테스트 이미지가 있는 폴더

In [ ]:
# optional
# YOLO 형식 바운딩 박스를 저장할 디렉토리
yolo_labels_dir = '/labels'
os.makedirs(yolo_labels_dir, exist_ok=True)

In [ ]:
visualize = False # 모델의 중간 레이어나 기능을 시각화할지 여부
augment = False # 예측 시에 데이터 증강(Data Augmentation)을 적용할지 여부를 설정
agnostic_nms = False # 클래스에 상관없이 비판별(non-class-specific) NMS를 수행할지 여부
# classes =  # (int | list[int], optional) 설명: 특정 클래스만 예측할지 설정하는 옵션
# embed = # (list[int], optional) 지정한 레이어로부터 **특징 벡터(embeddings)**를 추출할지 여부를 설정, 주로 분석용으로 사용

In [ ]:
# 예측 수행
results = model(test_images_path,
                imgsz = 1024, 
                iou = 0.6, 
                conf = 0.0001,
                visualize = visualize,
                augment = augment, 
                agnostic_nms = agnostic_nms
                #, classes =  # (int | list[int], optional) 설명: 특정 클래스만 예측할지 설정하는 옵션
                #, embed = # (list[int], optional) 지정한 레이어로부터 **특징 벡터(embeddings)**를 추출할지 여부를 설정, 주로 분석용으로 사용
                ) 

In [ ]:
# 예측 결과를 Pascal VOC 형식으로 변환하여 CSV 파일로 저장
rows = []
for result in results:
    image_id = result.path.split('/')[-1]  # 이미지 파일명
    image_name = os.path.splitext(image_id)[0]  # 확장자를 제거한 이미지 ID
    
    # image_id에 "test/" 경로 추가
    image_id = f"test/{image_id}" if not image_id.startswith("test/") else image_id
    
    predictions = result.boxes.xyxy.cpu().numpy()  # 바운딩 박스 좌표 (xmin, ymin, xmax, ymax)
    scores = result.boxes.conf.cpu().numpy()  # 신뢰도
    labels = result.boxes.cls.cpu().numpy()  # 클래스 라벨

    # YOLO 형식의 라벨을 저장할 텍스트 파일 경로
    yolo_label_file = os.path.join(yolo_labels_dir, f"{image_name}.txt")
    
    prediction_strings = []
    # 'w' 모드를 사용해 항상 덮어쓰기
    with open(yolo_label_file, 'w') as f:
        for label, score, (xmin, ymin, xmax, ymax) in zip(labels, scores, predictions):
            # YOLO 형식 (class_id, x_center, y_center, width, height)을 계산하기 위해 이미지 크기 정보 필요
            width = result.orig_shape[1]
            height = result.orig_shape[0]
            
            # YOLO 형식으로 변환 (정규화된 값 사용)
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            # YOLO 형식으로 텍스트 파일에 기록 (덮어쓰기)
            f.write(f"{int(label)} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")
            
            # Pascal VOC 형식으로 CSV 저장용 문자열 생성
            prediction_strings.append(f"{int(label)} {score:.6f} {xmin:.6f} {ymin:.6f} {xmax:.6f} {ymax:.6f}")

    # Pascal VOC 형식의 예측 문자열
    prediction_string = " ".join(prediction_strings)
    rows.append([prediction_string, image_id])

# 데이터프레임으로 변환하여 CSV로 저장
# 현재 시간 구하기
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f"{current_time}_submission.csv"

df = pd.DataFrame(rows, columns=['PredictionString', 'image_id'])
df.to_csv(csv_filename, index=False)

print(f"YOLO 형식의 바운딩 박스가 {yolo_labels_dir}에 저장되었습니다.")
print(f"CSV 파일이 {csv_filename}에 저장되었습니다.")